In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import time

table='table'

sql_conn_1 = pyodbc.connect(
    'DRIVER={SQL Server Native Client 11.0};'
    'SERVER=server_name;DATABASE=database_name;'
    'Trusted_Connection=yes'
)

sql_conn_2 = pyodbc.connect(
    'DRIVER={SQL Server Native Client 11.0};'
    'SERVER=server_name_2;DATABASE=database_name_2;'
    'Trusted_Connection=yes'
)

query = f'''SELECT *
      FROM database_name.{table} (NOLOCK)'''
print('Import from first_server...')
start = time.time()
df_first_server = pd.read_sql(query, sql_conn_1)
end = time.time()
print(f'Import from first server in:{(end-start)/60} minutes' )
print()    
print('Import from second server...')
start = time.time()      
df_second_server = pd.read_sql(query, sql_conn_2)
end = time.time()
print(f'Import from second server in:{(end-start)/60} minutes' )
print()
print('Mergeing...')
start = time.time()    
cols = [a for a in df_first_server.columns if a not in ['ID','generationDate','client_name']]
A = pd.merge(df_first_server,df_second_server, on='client_name', how='inner')
end = time.time()
print(f'Mergeing in:{(end-start)/60} minutes' )
print()
print('Loop...')
start = time.time()   
for col in cols:
    table=A.mask(A.eq('None')).dropna(subset=[col+'_x',col+'_y'])
    d = np.where((table[col+'_x'] != table[col+'_y']))
    print(col,np.shape(d)[1])
end = time.time()
print()
print(f'Lopp in:{(end-start)/60} minutes' )